# Example notebook to show how to use OGC API Processes

The necessary pygeoapi backend is available via https://github.com/52North/directed_pygeoapi_processes

In [ ]:
from io import BytesIO

import geopandas as gpd
import pandas as pd
import requests
#from owslib.ogcapi.processes import Processes

## Base configuration

In [ ]:
base_url = 'http://localhost:5000'
process_id = 'climada-simple-example-denmark-process'

## Helper class

In [ ]:
class ProcessesAPI():
    def __init__(self, url):
        self.base_url = url

    def execute_process(self, process_id: str, process_input: dict, headers: dict | None = None):
        url = f'{self.base_url}/processes/{process_id}/execution'
        response = requests.post(url=url, json=process_input, headers=headers)
        return response

    def get_job(self, job_id):
        url = f'{self.base_url}/jobs/{job_id}'
        response = requests.get(url)
        return response

    def get_job_result(self, job_id):
        url = f'{self.base_url}/jobs/{job_id}/results'
        try:
            response = requests.get(url)
            if response.headers['Content-Type'] == 'text/html':
                result = response.content
            elif response.headers['Content-Type'] == 'application/json':
                result = response.json()
            else:
                result = BytesIO(response.content)
        except Exception as err:
            print(f"Error: could't get result of job {job_id}: {err}")
        return result

## Execute a process

In [ ]:
api = ProcessesAPI(base_url)

Combinations

| climate_scenario | data_source | gcm |
| --- | --- | --- |
| ssp585 | CMIP6 | CMCC-ESM2 |
| ssp245 | CMIP6 | CanESM5 |
| ... | ... | ... |

In [ ]:
# Process configuration
async_execution = False

post_body = {
    "inputs": {
        "intensity": [0, 30, 80],
        "climate_scenario": "ssp585",
        "data_source": "CMIP6",
        "gcm": "CMCC-ESM2",
        "mdd": [0, 1],
        "paa": [1, 1],
        "nearest_neighbor_threshold": 1.5,
        "return_periods":  [1, 2, 5, 10, 20]
    },
    #"response": "raw",
    #"transmissionMode": "reference"
}

In [ ]:
if async_execution:
    headers = {
        "Prefer": "respond-async"
    }
else:
    headers = None

In [ ]:
execution_resp = api.execute_process(process_id, post_body, headers)

In [ ]:
print(execution_resp.status_code)
print(execution_resp.headers)

## Get process results

In [ ]:
# The GET {base_url}/jobs/{job_id}/results request is currently not working for binary output
#if execution_resp.status_code in [200, 201]:
#    job_id = execution_resp.headers['Location'].split('/')[-1]
#    job = api.get_job(job_id).json()
#    if job['status'] == 'successful':
#        result = api.get_job_result(job_id)
#    else:
#        print(f"Warning: processing job hasn't finished successfully, status = {job['status']}")
#else:
#    print(f"Warning: execution request failed with status code {execution_resp.status_code} and message {execution_resp.text}")

In [ ]:
result = BytesIO(execution_resp.content)

## Work with process results

In [ ]:
column_names = ['event_id', 'event_name', 'event_date', 'event_frequency', 'at_event', 'eai_exp', 'exp_lat', 'exp_lon']

In [ ]:
df = pd.read_csv(result, usecols=column_names)

In [ ]:
df.dropna(subset=['at_event'], inplace=True)

In [ ]:
df

In [ ]:
df[df['at_event'] != 0.0]

In [ ]:
df.plot(x='event_date', y='at_event')

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['exp_lat'], df['exp_lon']))

In [ ]:
gdf

In [ ]:
#gdf.plot()